# How to collect Datastream IBES Global Aggregate Earnings Data

In this article, we build the Python function ```Get_IBES_GA``` with [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html) [Dropdowns](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#Dropdown) to retrieve Institutional Brokers' Estimate System (IBES) Global Aggregate earnings data for country and regional sectors in an interactive way. With this function in CodeBook, no need to know how to code, it's as simple as clisk and play!


Returns a Pandas data-frame with:
* PE Ratio
* Earnings Growth
* PEG Ratio
* EPS
* Risk Premium (data not available for Regions)
* EV/EBITDA
* Revision Ratio
* Dividend Yield

Overview _Fiscal_Years 
* Ratios are displayed only for Fiscal Years (FY0, FY1, FY2, FY3)
* PE Ratio
* Earnings Growth
* PEG Ratio
* EPS
* Risk Premium (data not available for Regions, only for Countries)
* EV/EBITDA
* Revision Ratio
* Dividend Yield

## Contents

## Development Tools & Resources

The example code demonstrating the use case is based on the following development tools and resources:

* Refinitiv's [DataStream](https://www.refinitiv.com/en/products/datastream-macroeconomic-analysis) Web Services (DSWS): Access to DataStream data. A DataStream or Refinitiv Workspace IDentification (ID) will be needed to run the code below.

## Get to Coding

We need to gather our data. Since **Refinitiv's [DataStream](https://www.refinitiv.com/en/products/datastream-macroeconomic-analysis) Web Services (DSWS)** allows for access to ESG data covering nearly 70% of global market cap and over 400 metrics, naturally it is more than appropriate. We can access DSWS via the Python library "DatastreamDSWS" that can be installed simply by using $\textit{pip install}$.

In [1]:
import DatastreamDSWS as dsws

# We can use our Refinitiv's Datastream Web Socket (DSWS) API keys that allows us to be identified by Refinitiv's back-end services and enables us to request (and fetch) data: Credentials are placed in a text file so that it may be used in this code without showing it itself.
(dsws_username, dsws_password) = (open("Datastream_username.txt","r"),
                                  open("Datastream_password.txt","r"))

ds = dsws.Datastream(username = str(dsws_username.read()),
                     password = str(dsws_password.read()))

# It is best to close the files we opened in order to make sure that we don't stop any other services/programs from accessing them if they need to.
dsws_username.close()
dsws_password.close()


# # Alternatively one can use the following:
# import getpass
# dsusername = input()
# dspassword = getpass.getpass()
# ds = dsws.Datastream(username = dsusername, password = dspassword)

In [2]:
import warnings # ' warnings ' is a native Python library allowing us to raise warnings and errors to users.

In [79]:
from datetime import datetime # This is needed to keep track of when code runs.

In [3]:
import pandas as pd # pandas is an open source data analysis and manipulation tool that allows us to use data-frames. Version 1.1.4 was used in this article.

In [4]:
import numpy as np # NumPy is a library allowing for array manipulations including linear algebra, fourier transform, and matrices. Version 1.20.1 was used in this article.

In [59]:
from tqdm.notebook import trange # ' tqdm ' allows loops to show a progress meter. Version 4.48.2 was used in this article.

In [6]:
import ipywidgets as widgets # ipywidgets is a library of interactive HTML widgets for Jupyter notebooks and the IPython kernel. Version 7.5.1 was used in this article.
from IPython.display import display # This allows us to display data-frames.

In [7]:
for i,j in zip([pd, np, tqdm, widgets],["pandas", "numpy", "tqdm", "ipywidgets"]):
    print(f"The library {j} imported is version {i.__version__}")

The library pandas imported is version 1.1.4
The library numpy imported is version 1.20.1
The library tqdm imported is version 4.48.2
The library ipywidgets imported is version 7.5.1


## Step by Step Example

### Collecting Datastream Data <a class="anchor" id="collectingdatastreamdata"></a>

Users ought to be able to select their Country/Region of interest without knowing the Datastream Mnemonics for them. In line with this, the bellow creates tables of reference for Countries/Regions and such Mnemonics and Python functions to allow users easy data retreival.

##### Data-Frame of Countries, Regions and their Complimentary DSWS Mnemonic and IBES Code

In [8]:
# # From the ' Reference data.xls ' file, one could run and use the following:
# xl_countries = pd.read_excel("Reference data.xls", sheet_name = "Countries")
# xl_regions = pd.read_excel("Reference data.xls", sheet_name = "Regions")
# xl_index = pd.read_excel("Reference data.xls", sheet_name = "Index Names 3")
# xl_columns = pd.read_excel("Reference data.xls", sheet_name = "Column Names 2")

# # If you don't have the ' Reference data.xls ' file, just use the bellow:
xl_regions = pd.DataFrame(data = {'Region': {0: 'EAFE', 1: 'EAFE + Canada',2: 'EAFE-ex-UK',3: 'EASEA (EAFE-ex-Japan)',4: 'EM (Emerging Markets)',5: 'EM Asia',6: 'EM Eastern Europe',7: 'EM Europe',8: 'EM Europe + Middle East',9: 'EM Europe, Middle East & Africa',10: 'EM Far East',11: 'EM Latin America',12: 'EMU (Euro)',13: 'EMU (US Dollar)',14: 'EMU + UK',15: 'Europe',16: 'Europe-ex-EMU',17: 'Europe-ex-UK',18: 'Far East',19: 'G7 Index',20: 'Kokusai (World-ex-Japan)',21: 'Nordic Countries',22: 'North America',23: 'Pacific',24: 'Pacific-ex-Japan',25: 'World',26: 'World-ex-Australia',27: 'World-ex-EMU',28: 'World-ex-Europe',29: 'World-ex-UK',30: 'World-ex-USA'},'Mnemonic': {0: 'EA',1: 'FC',2: 'EX',3: 'AS',4: 'EF',5: 'MI',6: 'EB',7: 'ME',8: 'UE',9: 'EM',10: 'MP',11: 'ML',12: 'MU',13: 'M$',14: 'MK',15: 'UR',16: 'EE',17: 'UX',18: 'FE',19: 'G7',20: 'KK',21: 'NC',22: 'NA',23: 'PC',24: 'PX',25: 'DW',26: 'WA',27: 'WE',28: 'WC',29: 'WK',30: 'WS'},'Code': {0: '@:M1EAFE',1: '@:M1EAFEC',2: '@:M1EAFEU',3: '@:M1EASEA',4: '@:M2EMG',5: '@:M2ASIA',6: '@:M2EEUR',7: '@:M2EUR',8: '@:M2EURME',9: '@:M2EMEA',10: '@:M2FARE',11: '@:M2LATAM',12: '@:MUMSCI',13: '@:M1EMU',14: '@:MKMSCI',15: '@:M1EROP',16: '@:EEMSCI',17: '@:M1EURXU',18: '@:M1FARE',19: '@:G7MSCI',20: '@:M1KOKUS',21: '@:M1NORD',22: '@:M1NAMER',23: '@:M1PAC',24: '@:M1PACXJ',25: '@:M1WRLD',26: '@:WAMSCI',27: '@:WEMSCI',28: '@:WCMSCI',29: '@:M1WLDXU',30: '@:M1WLDXA'}})
xl_countries = pd.DataFrame(data = {'Region': {0: 'Argentina',1: 'Australia',2: 'Australia',3: 'Austria',4: 'Belgium',5: 'Brazil',6: 'Canada',7: 'Chile',8: 'China',9: 'Colombia',10: 'Czech Republic',11: 'Denmark',12: 'Egypt',13: 'Finland',14: 'France',15: 'Germany',16: 'Greece',17: 'Hong Kong',18: 'Hungary',19: 'India',20: 'Indonesia',21: 'Ireland',22: 'Israel',23: 'Italy',24: 'Japan',25: 'Jordan',26: 'Korea',27: 'Malaysia',28: 'Mexico',29: 'Morocco',30: 'Netherlands',31: 'New Zealand',32: 'Norway',33: 'Pakistan',34: 'Peru',35: 'Philippines',36: 'Poland',37: 'Portugal',38: 'Russia',39: 'Singapore',40: 'South Africa',41: 'Spain',42: 'Sri Lanka',43: 'Sweden',44: 'Switzerland',45: 'Taiwan',46: 'Thailand',47: 'Turkey',48: 'United Kingdom',49: 'USA',50: 'Venezuela'},'Mnemonic': {0: 'AR',1: 'AU',2: 'AU',3: 'AT',4: 'BE',5: 'BR',6: 'CA',7: 'CL',8: 'CN',9: 'CO',10: 'CZ',11: 'DK',12: 'EG',13: 'FI',14: 'FR',15: 'DE',16: 'GR',17: 'HK',18: 'HU',19: 'IN',20: 'ID',21: 'IE',22: 'IL',23: 'IT',24: 'JP',25: 'JO',26: 'KR',27: 'MY',28: 'MX',29: 'MA',30: 'NL',31: 'NZ',32: 'NO',33: 'PK',34: 'PE',35: 'PH',36: 'PL',37: 'PT',38: 'RU',39: 'SG',40: 'SA',41: 'ES',42: 'LK',43: 'SE',44: 'SW',45: 'TW',46: 'TH',47: 'TR',48: 'UK',49: 'US',50: 'VE'}, 'Code': {0: '@:ARMSCIP',1: '@:MSCIP',2: '@:AUMSCIP',3: '@:OEMSCIP',4: '@:BGMSCIP',5: '@:BRMSCIP',6: '@:CNMSCIP',7: '@:CLMSCIP',8: '@:CHMSCIP',9: '@:CBMSCIP',10: '@:CZMSCIP',11: '@:DKMSCIP',12: '@:EYMSCIP',13: '@:FNMSCIP',14: '@:FRMSCIP',15: '@:BDMSCIP',16: '@:GRMSCIP',17: '@:HKMSCIP',18: '@:HNMSCIP',19: '@:INMSCIP',20: '@:IDMSCIP',21: '@:IRMSCIP',22: '@:ISMSCIP',23: '@:ITMSCIP',24: '@:JPMSCIP',25: '@:JOMSCIP',26: '@:KOMSCIP',27: '@:MYMSCIP',28: '@:MXMSCIP',29: '@:MCMSCIP',30: '@:NLMSCIP',31: '@:NZMSCIP',32: '@:NWMSCIP',33: '@:PKMSCIP',34: '@:PEMSCIP',35: '@:PHMSCIP',36: '@:POMSCIP',37: '@:PTMSCIP',38: '@:RSMSCIP',39: '@:SGMSCIP',40: '@:SAMSCIP',41: '@:ESMSCIP',42: '@:CYMSCIP',43: '@:SDMSCIP',44: '@:SWMSCIP',45: '@:TAMSCIP',46: '@:THMSCIP',47: '@:TKMSCIP',48: '@:UKMSCIP',49: '@:USMSCIP',50: '@:VEMSCIP'}})
# We will use the following to index and column our final data-frame on interest:
xl_index = pd.DataFrame(data = {'Category': {0: 'Energy', 1: 'Energy', 2: 'Energy', 3: 'Energy', 4: 'Materials', 5: 'Materials', 6: 'Materials', 7: 'Materials', 8: 'Materials', 9: 'Materials', 10: 'Materials', 11: 'Industrials', 12: 'Industrials', 13: 'Industrials', 14: 'Industrials', 15: 'Industrials', 16: 'Industrials', 17: 'Industrials', 18: 'Industrials', 19: 'Industrials', 20: 'Industrials', 21: 'Industrials', 22: 'Industrials', 23: 'Industrials', 24: 'Industrials', 25: 'Industrials', 26: 'Industrials', 27: 'Industrials', 28: 'Consumer Discretionary', 29: 'Consumer Discretionary', 30: 'Consumer Discretionary', 31: 'Consumer Discretionary', 32: 'Consumer Discretionary', 33: 'Consumer Discretionary', 34: 'Consumer Discretionary', 35: 'Consumer Discretionary', 36: 'Consumer Discretionary', 37: 'Consumer Discretionary', 38: 'Consumer Discretionary', 39: 'Consumer Discretionary', 40: 'Consumer Discretionary', 41: 'Consumer Discretionary', 42: 'Consumer Discretionary', 43: 'Consumer Discretionary', 44: 'Consumer Discretionary', 45: 'Consumer Staples', 46: 'Consumer Staples', 47: 'Consumer Staples', 48: 'Consumer Staples', 49: 'Consumer Staples', 50: 'Consumer Staples', 51: 'Consumer Staples', 52: 'Consumer Staples', 53: 'Consumer Staples', 54: 'Consumer Staples', 55: 'Health Care', 56: 'Health Care', 57: 'Health Care', 58: 'Health Care', 59: 'Health Care', 60: 'Health Care', 61: 'Health Care', 62: 'Financials', 63: 'Financials', 64: 'Financials', 65: 'Financials', 66: 'Financials', 67: 'Financials', 68: 'Financials', 69: 'Real Estate', 70: 'Information Technology', 71: 'Information Technology', 72: 'Information Technology', 73: 'Information Technology', 74: 'Information Technology', 75: 'Information Technology', 76: 'Information Technology', 77: 'Information Technology', 78: 'Information Technology', 79: 'Information Technology', 80: 'Telecommunication Services', 81: 'Telecommunication Services', 82: 'Telecommunication Services', 83: 'Telecommunication Services', 84: 'Utilities', 85: 'Utilities', 86: 'Utilities', 87: 'Utilities', 88: 'Utilities', 89: 'Utilities'}, 'Sub-Category': {0: 'Energy', 1: 'Energy', 2: 'Energy', 3: 'Energy', 4: 'Materials', 5: 'Materials', 6: 'Materials', 7: 'Materials', 8: 'Materials', 9: 'Materials', 10: 'Materials', 11: 'Industrials', 12: 'Capital Goods', 13: 'Capital Goods', 14: 'Capital Goods', 15: 'Capital Goods', 16: 'Capital Goods', 17: 'Capital Goods', 18: 'Capital Goods', 19: 'Capital Goods', 20: 'Commercial Services & Supplies', 21: 'Commercial Services & Supplies', 22: 'Transportation', 23: 'Transportation', 24: 'Transportation', 25: 'Transportation', 26: 'Transportation', 27: 'Transportation', 28: 'Consumer Discretionary', 29: 'Automobiles & Components', 30: 'Automobiles & Components', 31: 'Automobiles & Components', 32: 'Consumer Durables & Apparel', 33: 'Consumer Durables & Apparel', 34: 'Consumer Durables & Apparel', 35: 'Consumer Durables & Apparel', 36: 'Hotels, Restaurants & Leisure', 37: 'Hotels, Restaurants & Leisure', 38: 'Media', 39: 'Media', 40: 'Retailing', 41: 'Retailing', 42: 'Retailing', 43: 'Retailing', 44: 'Retailing', 45: 'Consumer Staples', 46: 'Food & Staples Retailing', 47: 'Food & Staples Retailing', 48: 'Food Beverage & Tobacco', 49: 'Food Beverage & Tobacco', 50: 'Food Beverage & Tobacco', 51: 'Food Beverage & Tobacco', 52: 'Household & Personal Products', 53: 'Household & Personal Products', 54: 'Household & Personal Products', 55: 'Health Care', 56: 'Health Care Equipment & Services', 57: 'Health Care Equipment & Services', 58: 'Health Care Equipment & Services', 59: 'Pharmaceuticals, Biotechnology & Life Sciences', 60: 'Pharmaceuticals, Biotechnology & Life Sciences', 61: 'Pharmaceuticals, Biotechnology & Life Sciences', 62: 'Financials', 63: 'Banks', 64: 'Banks', 65: 'Diversified Financials', 66: 'Diversified Financials', 67: 'Insurance', 68: 'Insurance', 69: 'Real Estate', 70: 'Information Technology', 71: 'Software & Services', 72: 'Software & Services', 73: 'Software & Services', 74: 'Software & Services', 75: 'Technology Hardware & Equipment', 76: 'Technology Hardware & Equipment', 77: 'Technology Hardware & Equipment', 78: 'Technology Hardware & Equipment', 79: 'Technology Hardware & Equipment', 80: 'Telecommunication Services', 81: 'Telecommunication Services', 82: 'Telecommunication Services', 83: 'Telecommunication Services', 84: 'Utilities', 85: 'Utilities', 86: 'Utilities', 87: 'Utilities', 88: 'Utilities', 89: 'Utilities'}, 'Sector': {0: 'Energy', 1: 'Energy', 2: 'Energy Equipment & Services ', 3: 'Oil, Gas & Consumable Fuels ', 4: 'Materials', 5: 'Materials', 6: 'Chemicals ', 7: 'Construction Materials ', 8: 'Containers & Packaging ', 9: 'Metals & Mining ', 10: 'Paper & Forest Products ', 11: 'Industrials', 12: 'Capital Goods', 13: 'Aerospace & Defense ', 14: 'Building Products ', 15: 'Construction & Engineering ', 16: 'Electronic Equipment & Instruments', 17: 'Industrial Conglomerates ', 18: 'Machinery ', 19: 'Trading Companies & Distributors Industry', 20: 'Commercial Services & Supplies', 21: 'Commercial Services & Supplies ', 22: 'Transportation', 23: 'Air Freight & Couriers ', 24: 'Airlines ', 25: 'Marine ', 26: 'Road & Rail ', 27: 'Transportation Infrastructure ', 28: 'Consumer Discretionary', 29: 'Automobiles & Components', 30: 'Auto Components ', 31: 'Automobiles ', 32: 'Consumer Durables & Apparel', 33: 'Household Durables ', 34: 'Leisure Equipment & Products ', 35: 'Textiles & Apparel ', 36: 'Hotels, Restaurants & Leisure', 37: 'Hotels Restaurants & Leisure ', 38: 'Media', 39: 'Media ', 40: 'Retailing', 41: 'Distributors ', 42: 'Internet & Catalog Retail ', 43: 'Multiline Retail ', 44: 'Specialty Retail ', 45: 'Consumer Staples', 46: 'Food & Staples Retailing', 47: 'Food & Staples Retailing ', 48: 'Food Beverage & Tobacco', 49: 'Beverages ', 50: 'Food Products ', 51: 'Tobacco ', 52: 'Household & Personal Products', 53: 'Household Products ', 54: 'Personal Products ', 55: 'Health Care', 56: 'Health Care Equipment & Services', 57: 'Health Care Equipment & Supplies', 58: 'Health Care Providers & Services', 59: 'Pharmaceuticals, Biotechnology & Life Sciences', 60: 'Biotechnology ', 61: 'Pharmaceuticals ', 62: 'Financials', 63: 'Banks', 64: 'Banks', 65: 'Diversified Financials', 66: 'Diversified Financial Services ', 67: 'Insurance', 68: 'Insurance ', 69: 'Real Estate', 70: 'Information Technology', 71: 'Software & Services', 72: 'Internet Software & Services ', 73: 'IT Consulting & Services ', 74: 'Software ', 75: 'Technology Hardware & Equipment', 76: 'Communications Equipment ', 77: 'Computers & Peripherals ', 78: 'Electrical Equipment ', 79: 'Semiconductors & Semiconductor Equipment ', 80: 'Telecommunication Services', 81: 'Telecommunication Services', 82: 'Diversified Telecommunications Services   ', 83: 'Wireless Telecommunication Services', 84: 'Utilities', 85: 'Utilities', 86: 'Electric Utilities ', 87: 'Gas Utilities ', 88: 'Multi-Utilities ', 89: 'Water Utilities '}, 'Mnemonic': {0: 'M1E1', 1: 'M2E2', 2: 'M3ES', 3: 'M3OG', 4: 'M1M1', 5: 'M2M2', 6: 'M3CH', 7: 'M3CM', 8: 'M3CT', 9: 'M3MM', 10: 'M3PF', 11: 'M1ID', 12: 'M2CG', 13: 'M3AD', 14: 'M3BP', 15: 'M3CN', 16: 'M3EI', 17: 'M3IC', 18: 'M3MC', 19: 'M3TC', 20: 'M2C2', 21: 'M3C3', 22: 'M2TR', 23: 'M3AF', 24: 'M3AL', 25: 'M3MA', 26: 'M3RR', 27: 'M3TI', 28: 'M1CD', 29: 'M2AC', 30: 'M3AU', 31: 'M3AM', 32: 'M2CA', 33: 'M3HD', 34: 'M3LE', 35: 'M3TA', 36: 'M2HR', 37: 'M3HR', 38: 'M2MD', 39: 'M3ME', 40: 'M2RT', 41: 'M3DI', 42: 'M3NT', 43: 'M3MR', 44: 'M3SR', 45: 'M1CS', 46: 'M2FD', 47: 'M3FD', 48: 'M2FB', 49: 'M3BV', 50: 'M3FP', 51: 'M3TB', 52: 'M2HH', 53: 'M3HP', 54: 'M3PP', 55: 'M1HC', 56: 'M2HE', 57: 'M3HS', 58: 'M3PS', 59: 'M2PB', 60: 'M3BI', 61: 'M3PH', 62: 'M1FN', 63: 'M2B2', 64: 'M3B3', 65: 'M2D2', 66: 'M3D3', 67: 'M2I2', 68: 'M3I3', 69: 'M1RE', 70: 'M1IT', 71: 'M2SS', 72: 'M3NS', 73: 'M3IS', 74: 'M3SW', 75: 'M2TH', 76: 'M3CE', 77: 'M3CP', 78: 'M3EE', 79: 'M3SC', 80: 'M1T1', 81: 'M2T2', 82: 'M3DT', 83: 'M3WT', 84: 'M1U1', 85: 'M2U2', 86: 'M3EU', 87: 'M3GU', 88: 'M3MU', 89: 'M3WU'}})
xl_columns = pd.DataFrame({'Column 1': {0: 'PE-Ratio', 1: 'PE-Ratio', 2: 'PE-Ratio', 3: 'PE-Ratio', 4: 'PE-Ratio', 5: 'PE-Ratio', 6: 'Earnings Growth', 7: 'Earnings Growth', 8: 'Earnings Growth', 9: 'Earnings Growth', 10: 'PEG-Ratio', 11: 'PEG-Ratio', 12: 'PEG-Ratio', 13: 'PEG-Ratio', 14: 'EPS Mean', 15: 'EPS Mean', 16: 'EPS Mean', 17: 'EPS Mean', 18: 'EPS Mean', 19: 'EPS Mean', 20: 'Risk premium', 21: 'Risk premium', 22: 'Risk premium', 23: 'Risk premium', 24: 'Risk premium', 25: 'Risk premium', 26: 'EV/EBITDA', 27: 'EV/EBITDA', 28: 'EV/EBITDA', 29: 'EV/EBITDA', 30: 'EV/EBITDA', 31: 'EV/EBITDA', 32: 'Rev. Ratio', 33: 'Div Yield'}, 'Column 2': {0: 'Fiscal Year', 1: 'Fiscal Year', 2: 'Fiscal Year', 3: 'Fiscal Year', 4: 'Forward', 5: 'Forward', 6: 'Fiscal Year', 7: 'Fiscal Year', 8: 'Fiscal Year', 9: 'Fiscal Year', 10: 'Fiscal Year', 11: 'Fiscal Year', 12: 'Fiscal Year', 13: 'Fiscal Year', 14: 'Fiscal Year', 15: 'Fiscal Year', 16: 'Fiscal Year', 17: 'Fiscal Year', 18: 'Forward', 19: 'Forward', 20: 'Fiscal Year', 21: 'Fiscal Year', 22: 'Fiscal Year', 23: 'Fiscal Year', 24: 'Forward', 25: 'Forward', 26: 'Fiscal Year', 27: 'Fiscal Year', 28: 'Fiscal Year', 29: 'Fiscal Year', 30: 'Forward', 31: 'Forward', 32: '', 33: ''}, 'Column 3': {0: 'FY0', 1: 'FY1', 2: 'FY2', 3: 'FY3', 4: '12M', 5: '18M', 6: 'FY0', 7: 'FY1', 8: 'FY2', 9: 'FY3', 10: 'FY0', 11: 'FY1', 12: 'FY2', 13: 'FY3', 14: 'FY0', 15: 'FY1', 16: 'FY2', 17: 'FY3', 18: '12M', 19: '18M', 20: 'FY0', 21: 'FY1', 22: 'FY2', 23: 'FY3', 24: '12M', 25: '18M', 26: 'FY0', 27: 'FY1', 28: 'FY2', 29: 'FY3', 30: '12M', 31: '18M', 32: '', 33: ''}})

# # Merging the two data-frames together:
xl_regions = xl_regions.append(xl_countries, ignore_index = True)

#### Python function ```Region_to_DSWS_region_mnemonic_and_IBES_code```

As per its written definition bellow, this function returns a string of the Datastream Web Service (DSWS) region's mnemonic and IBES code. For certain requests, DSWS needs a ticker specified with a nomenclature that includes DSWS region's mnemonic.

In [9]:
def Region_to_DSWS_region_mnemonic_and_IBES_code(region = None, xl_regions = None):
    """Region_to_DSWS_region_mnemonic_and_IBES_code Version 1.0:
    This function returns a string of the Datastream Web Service (DSWS) region's mnemonic and IBES code.
    DSWS is Refinitiv's API retreaving data from Datastream to a Python Pandas data-frame. For information on DSWS, please visit 'https://developers.refinitiv.com/en/api-catalog/eikon/datastream-web-service'.
    For certain requests, DSWS needs a ticker specified with a nomenclature that includes DSWS region's mnemonic. For an example, see 'Examples' bellow.    
    
    
    Parameters:
    ----------
    
    region: str
        Region of choice's name (e.g.: 'EAFE-ex-UK').
        It has to be one of the elements in the following list: ['EAFE' 'EAFE + Canada' 'EAFE-ex-UK' 'EASEA (EAFE-ex-Japan)'  'EM (Emerging Markets)' 'EM Asia' 'EM Eastern Europe' 'EM Europe'  'EM Europe + Middle East' 'EM Europe, Middle East & Africa' 'EM Far East'  'EM Latin America' 'EMU (Euro)' 'EMU (US Dollar)' 'EMU + UK' 'Europe'  'Europe-ex-EMU' 'Europe-ex-UK' 'Far East' 'G7 Index'  'Kokusai (World-ex-Japan)' 'Nordic Countries' 'North America' 'Pacific'  'Pacific-ex-Japan' 'World' 'World-ex-Australia' 'World-ex-EMU'  'World-ex-Europe' 'World-ex-UK' 'World-ex-USA' 'Argentina' 'Australia'  'Australia' 'Austria' 'Belgium' 'Brazil' 'Canada' 'Chile' 'China'  'Colombia' 'Czech Republic' 'Denmark' 'Egypt' 'Finland' 'France'  'Germany' 'Greece' 'Hong Kong' 'Hungary' 'India' 'Indonesia' 'Ireland'  'Israel' 'Italy' 'Japan' 'Jordan' 'Korea' 'Malaysia' 'Mexico' 'Morocco'  'Netherlands' 'New Zealand' 'Norway' 'Pakistan' 'Peru' 'Philippines'  'Poland' 'Portugal' 'Russia' 'Singapore' 'South Africa' 'Spain'  'Sri Lanka' 'Sweden' 'Switzerland' 'Taiwan' 'Thailand' 'Turkey'  'United Kingdom' 'USA' 'Venezuela']
        Default: region = None
    
    xl_regions: Pandas data-frame
        The data-frame of regions to map with, including columns 'Region', 'Mnemonic' and 'Code'.
        This function (' Region_to_DSWS_region_mnemonic_and_IBES_code ') will map ' region ' to 'Mnemonic' and 'Code' in this data-frame and return them.
        If ' None ', a pre-defined data-frame is used.
        It is named with 'xl' at its start because it originally came from the Excel workbook 'Datastream IBES Global Aggregates MSCI.xlsm'.
        Default: xl_regions = None
    
    
    Dependencies:
    ----------
    
    pandas 1.0.3
    
    
    Examples:
    --------
    
    >>> Region_to_DSWS_region_mnemonic_and_IBES_code("EAFE + Canada")
    ('FC', '@:M1EAFEC')
    """
    if xl_regions == None: xl_regions = pd.DataFrame(data = {'Region': {0: 'EAFE', 1: 'EAFE + Canada', 2: 'EAFE-ex-UK', 3: 'EASEA (EAFE-ex-Japan)', 4: 'EM (Emerging Markets)', 5: 'EM Asia', 6: 'EM Eastern Europe', 7: 'EM Europe', 8: 'EM Europe + Middle East', 9: 'EM Europe, Middle East & Africa', 10: 'EM Far East', 11: 'EM Latin America', 12: 'EMU (Euro)', 13: 'EMU (US Dollar)', 14: 'EMU + UK', 15: 'Europe', 16: 'Europe-ex-EMU', 17: 'Europe-ex-UK', 18: 'Far East', 19: 'G7 Index', 20: 'Kokusai (World-ex-Japan)', 21: 'Nordic Countries', 22: 'North America', 23: 'Pacific', 24: 'Pacific-ex-Japan', 25: 'World', 26: 'World-ex-Australia', 27: 'World-ex-EMU', 28: 'World-ex-Europe', 29: 'World-ex-UK', 30: 'World-ex-USA', 31: 'Argentina', 32: 'Australia', 33: 'Australia', 34: 'Austria', 35: 'Belgium', 36: 'Brazil', 37: 'Canada', 38: 'Chile', 39: 'China', 40: 'Colombia', 41: 'Czech Republic', 42: 'Denmark', 43: 'Egypt', 44: 'Finland', 45: 'France', 46: 'Germany', 47: 'Greece', 48: 'Hong Kong', 49: 'Hungary', 50: 'India', 51: 'Indonesia', 52: 'Ireland', 53: 'Israel', 54: 'Italy', 55: 'Japan', 56: 'Jordan', 57: 'Korea', 58: 'Malaysia', 59: 'Mexico', 60: 'Morocco', 61: 'Netherlands', 62: 'New Zealand', 63: 'Norway', 64: 'Pakistan', 65: 'Peru', 66: 'Philippines', 67: 'Poland', 68: 'Portugal', 69: 'Russia', 70: 'Singapore', 71: 'South Africa', 72: 'Spain', 73: 'Sri Lanka', 74: 'Sweden', 75: 'Switzerland', 76: 'Taiwan', 77: 'Thailand', 78: 'Turkey', 79: 'United Kingdom', 80: 'USA', 81: 'Venezuela'}, 'Mnemonic': {0: 'EA', 1: 'FC', 2: 'EX', 3: 'AS', 4: 'EF', 5: 'MI', 6: 'EB', 7: 'ME', 8: 'UE', 9: 'EM', 10: 'MP', 11: 'ML', 12: 'MU', 13: 'M$', 14: 'MK', 15: 'UR', 16: 'EE', 17: 'UX', 18: 'FE', 19: 'G7', 20: 'KK', 21: 'NC', 22: 'NA', 23: 'PC', 24: 'PX', 25: 'DW', 26: 'WA', 27: 'WE', 28: 'WC', 29: 'WK', 30: 'WS', 31: 'AR', 32: 'AU', 33: 'AU', 34: 'AT', 35: 'BE', 36: 'BR', 37: 'CA', 38: 'CL', 39: 'CN', 40: 'CO', 41: 'CZ', 42: 'DK', 43: 'EG', 44: 'FI', 45: 'FR', 46: 'DE', 47: 'GR', 48: 'HK', 49: 'HU', 50: 'IN', 51: 'ID', 52: 'IE', 53: 'IL', 54: 'IT', 55: 'JP', 56: 'JO', 57: 'KR', 58: 'MY', 59: 'MX', 60: 'MA', 61: 'NL', 62: 'NZ', 63: 'NO', 64: 'PK', 65: 'PE', 66: 'PH', 67: 'PL', 68: 'PT', 69: 'RU', 70: 'SG', 71: 'SA', 72: 'ES', 73: 'LK', 74: 'SE', 75: 'SW', 76: 'TW', 77: 'TH', 78: 'TR', 79: 'UK', 80: 'US', 81: 'VE'}, 'Code': {0: '@:M1EAFE', 1: '@:M1EAFEC', 2: '@:M1EAFEU', 3: '@:M1EASEA', 4: '@:M2EMG', 5: '@:M2ASIA', 6: '@:M2EEUR', 7: '@:M2EUR', 8: '@:M2EURME', 9: '@:M2EMEA', 10: '@:M2FARE', 11: '@:M2LATAM', 12: '@:MUMSCI', 13: '@:M1EMU', 14: '@:MKMSCI', 15: '@:M1EROP', 16: '@:EEMSCI', 17: '@:M1EURXU', 18: '@:M1FARE', 19: '@:G7MSCI', 20: '@:M1KOKUS', 21: '@:M1NORD', 22: '@:M1NAMER', 23: '@:M1PAC', 24: '@:M1PACXJ', 25: '@:M1WRLD', 26: '@:WAMSCI', 27: '@:WEMSCI', 28: '@:WCMSCI', 29: '@:M1WLDXU', 30: '@:M1WLDXA', 31: '@:ARMSCIP', 32: '@:MSCIP', 33: '@:AUMSCIP', 34: '@:OEMSCIP', 35: '@:BGMSCIP', 36: '@:BRMSCIP', 37: '@:CNMSCIP', 38: '@:CLMSCIP', 39: '@:CHMSCIP', 40: '@:CBMSCIP', 41: '@:CZMSCIP', 42: '@:DKMSCIP', 43: '@:EYMSCIP', 44: '@:FNMSCIP', 45: '@:FRMSCIP', 46: '@:BDMSCIP', 47: '@:GRMSCIP', 48: '@:HKMSCIP', 49: '@:HNMSCIP', 50: '@:INMSCIP', 51: '@:IDMSCIP', 52: '@:IRMSCIP', 53: '@:ISMSCIP', 54: '@:ITMSCIP', 55: '@:JPMSCIP', 56: '@:JOMSCIP', 57: '@:KOMSCIP', 58: '@:MYMSCIP', 59: '@:MXMSCIP', 60: '@:MCMSCIP', 61: '@:NLMSCIP', 62: '@:NZMSCIP', 63: '@:NWMSCIP', 64: '@:PKMSCIP', 65: '@:PEMSCIP', 66: '@:PHMSCIP', 67: '@:POMSCIP', 68: '@:PTMSCIP', 69: '@:RSMSCIP', 70: '@:SGMSCIP', 71: '@:SAMSCIP', 72: '@:ESMSCIP', 73: '@:CYMSCIP', 74: '@:SDMSCIP', 75: '@:SWMSCIP', 76: '@:TAMSCIP', 77: '@:THMSCIP', 78: '@:TKMSCIP', 79: '@:UKMSCIP', 80: '@:USMSCIP', 81: '@:VEMSCIP'}})
    
    if region not in xl_regions["Region"].to_list():
        print("Invalid ' region ' argument specified.")
    else:
        return xl_regions[xl_regions["Region"] == region]["Mnemonic"].values[0], xl_regions[xl_regions["Region"] == region]["Code"].values[0]

For example, we could get the Mnemonic and Code for the United Kingdom:

In [10]:
Region_to_DSWS_region_mnemonic_and_IBES_code("United Kingdom")

('UK', '@:UKMSCIP')

#### Example with Region "EAFE + Canada"

We 1st need to setup baseline variables

In [11]:
ordered_mnemonic = "M1E1,M2E2,M3ES,M3OG,M1M1,M2M2,M3CH,M3CM,M3CT,M3MM,M3PF,M1ID,M2CG,M3AD,M3BP,M3CN,M3EI,M3IC,M3MC,M3TC,M2C2,M3C3,M2TR,M3AF,M3AL,M3MA,M3RR,M3TI,M1CD,M2AC,M3AU,M3AM,M2CA,M3HD,M3LE,M3TA,M2HR,M3HR,M2MD,M3ME,M2RT,M3DI,M3NT,M3MR,M3SR,M1CS,M2FD,M3FD,M2FB,M3BV,M3FP,M3TB,M2HH,M3HP,M3PP,M1HC,M2HE,M3HS,M3PS,M2PB,M3BI,M3PH,M1FN,M2B2,M3B3,M2D2,M3D3,M2I2,M3I3,M1RE,M1IT,M2SS,M3NS,M3IS,M3SW,M2TH,M3CE,M3CP,M3EE,M3SC,M1T1,M2T2,M3DT,M3WT,M1U1,M2U2,M3EU,M3GU,M3MU,M3WU"

# The long string for ' IBESGA_fields ' (and subsequently ' IBESGA_tickers_str ') can be found in the ' Datastream IBES Global Aggregates MSCI .xlsm ' file.
IBESGA_fields = "ALNAME,AF0PE,AF1PE,AF2PE,AF3PE,A12PE,A18PE,AF0GRO,AF1GRO,AF2GRO,AF3GRO,AF0VAL,AF1MN,AF2MN,AF3MN,A12FE,A18FE,1/X(AF0PE)*100-X(AGBYLD),1/X(AF1PE)*100-X(AGBYLD),1/X(AF2PE)*100-X(AGBYLD),1/X(AF3PE)*100-X(AGBYLD),1/X(A12PE)*100-X(AGBYLD),1/X(A18PE)*100-X(AGBYLD),X(AZ0VAL)/X(AT0VAL),X(AZ1MN)/X(AT1MN),X(AZ2MN)/X(AT2MN),X(AZ3MN)/X(AT3MN),X(AZ12FZ)/X(AT12FT),X(AZ18FZ)/X(AT18FT),429E*1.0000,ADVYLD"
IBESGA_fields = IBESGA_fields.split(",")

# Callin it with ' _str ' at the end to disosiate it from other objects and use it later:
IBESGA_tickers_str = "M1CD,M1CS,M1E1,M1FN,M1HC,M1ID,M1IT,M1M1,M1T1,M1U1,M1RE,M2AC,M2B2,M2C2,M2CA,M2CG,M2D2,M2E2,M2FB,M2FD,M2HE,M2HH,M2HR,M2I2,M2M2,M2MD,M2PB,M2RT,M2SS,M2T2,M2TH,M2TR,M2U2,M3AD,M3AF,M3AL,M3AM,M3AU,M3B3,M3BI,M3BP,M3BV,M3C3,M3CE,M3CH,M3CM,M3CN,M3CP,M3CT,M3D3,M3DI,M3DT,M3EE,M3EI,M3ES,M3EU,M3FD,M3FP,M3GU,M3HD,M3HP,M3HR,M3HS,M3I3,M3IC,M3IS,M3LE,M3MA,M3MC,M3ME,M3MM,M3MR,M3MU,M3NS,M3NT,M3OG,M3PF,M3PH,M3PP,M3PS,M3RR,M3SC,M3SR,M3SW,M3TA,M3TB,M3TC,M3TI,M3WT,M3WU"

IBESGA_full_tickers = ["@:" + Region_to_DSWS_region_mnemonic_and_IBES_code("EAFE + Canada")[0] + i for i in IBESGA_tickers_str.split(",")]

Now we can go ahead and collect our data from Datastream. Note that we split requests in batches using ```ds_get_twice_data``` to keep well within request limits.

In [12]:
# Defined our dsws data retrieval function
def ds_get_twice_data(tickers, fields, batch = 15, kind = 0):
    
    df = ds.get_data(tickers = tickers[0],
                     fields = fields[:batch],
                     kind = kind)
    _df = ds.get_data(tickers = tickers[0],
                      fields = fields[batch:],
                      kind = kind)
    df = df.append(_df, ignore_index = True)
    
    for i in tickers[1:]:
        _df1 = ds.get_data(tickers = i,
                          fields = fields[:batch],
                           kind = kind)
        _df2 = ds.get_data(tickers = i,
                          fields = fields[batch:],
                          kind = kind)
        _df = _df1.append(_df2, ignore_index = True)
        df = df.append(_df, ignore_index = True)
    
    return df

In [13]:
# Collect our data info data-frame ' df ':
df = ds_get_twice_data(tickers = IBESGA_full_tickers, fields = IBESGA_fields, batch = 15, kind = 0)

# Tidy our ' df ' and replace stings 'NA' with computationally recognisable nan values:
df = pd.DataFrame(
    index = df["Instrument"].unique(), columns = df["Datatype"].unique(),
    data = [list(df["Value"][df["Instrument"] == i]) for i in df["Instrument"].unique()]).replace('NA', np.nan, regex=True)

# Let's keep the tiker names:
df["Tickers"] = df.index
df.index = IBESGA_tickers_str.split(",")
df = df.T[ordered_mnemonic.split(",")].T

# Now re-index the data-frame:
_index = [list(xl_index.fillna(method='ffill').loc[i].values) for i in range(len(xl_index.index))]
df.index = pd.MultiIndex.from_tuples(_index)

In [14]:
# # re-column the data-frame:

# We need to add a column named 'ALNAME'. Since it's a column with 3 levels, it needs to be added thrice.
xl_columns2 = pd.concat([pd.DataFrame({'Column 1': {0: 'ALNAME'}, 'Column 2': {0: ''}, 'Column 3': {0: ''}}), xl_columns], ignore_index = True)
# We need to add a column named 'Tickers' similarlly.
xl_columns2 = pd.concat([xl_columns2, pd.DataFrame({'Column 1': {0: 'Tickers'}, 'Column 2': {0: ''}, 'Column 3': {0: ''}})], ignore_index = True)
# Collumns 'PEG-Ratio' 'Fiscal Year', 'FY0' to 'FY3' happen to be calculated, not pulled from DSWS, so we need to ignore than mor now and add them later
xl_columns3 = xl_columns2.drop([11,12,13,14]).reset_index(drop = True)

_columns = [list(xl_columns3.fillna(method='ffill').loc[i].values) for i in range(len(xl_columns3.index))]
df.columns = pd.MultiIndex.from_tuples(_columns)

In [15]:
# PEG-Ratio columns need to be calculated:
for i in range(4):
    df["PEG-Ratio", "Fiscal Year", f"FY{i}"] = df["PE-Ratio", "Fiscal Year", f"FY{i}"] / df["Earnings Growth", "Fiscal Year", f"FY{i}"]

Now let's see our data-frame:

In [16]:
df

ALNAME  \
                                                                                                           
                                                                                                           
Energy    Energy    Energy                       M1E1                   MSCI EAFE + Canada Energy Sector   
                                                 M2E2           MSCI EAFE + Canada Energy Industry Group   
                    Energy Equipment & Services  M3ES  MSCI EAFE + Canada Energy Equipment & Services...   
                    Oil, Gas & Consumable Fuels  M3OG  MSCI EAFE + Canada Oil, Gas & Consumable Fuels...   
Materials Materials Materials                    M1M1                MSCI EAFE + Canada Materials Sector   
...                                                                                                  ...   
Utilities Utilities Utilities                    M2U2        MSCI EAFE + Canada Utilities Industry Group   
                    Electric Utilities           M3EU     MSCI EAFE + Canada Electric Utilities Industry   
                    Gas Utilities                M3GU          MSCI EAFE + Canada Gas Utilities Industry   
                    Multi-Utilities              M3MU        MSCI EAFE + Canada Multi-Utilities Industry   
                    Water Utilities              M3WU        MSCI EAFE + Canada Water Utilities Industry   

                                                         PE-Ratio          \
                                                      Fiscal Year           
                                                              FY0     FY1   
Energy    Energy    Energy                       M1E1      79.201  12.338   
                                                 M2E2      79.201  12.338   
                    Energy Equipment & Services  M3ES         NaN  36.341   
                    Oil, Gas & Consumable Fuels  M3OG      78.641  12.284   
Materials Materials Materials                    M1M1      22.863  13.408   
...                                                           ...     ...   
Utilities Utilities Utilities                    M2U2      18.152  17.298   
                    Electric Utilities           M3EU      17.176  17.407   
                    Gas Utilities                M3GU      19.641  18.267   
                    Multi-Utilities              M3MU      19.432  15.926   
                    Water Utilities              M3WU      20.794  19.466   

                                                                               \
                                                                      Forward   
                                                          FY2     FY3     12M   
Energy    Energy    Energy                       M1E1  10.531   9.848  11.675   
                                                 M2E2  10.531   9.848  11.675   
                    Energy Equipment & Services  M3ES  28.034  21.719  33.074   
                    Oil, Gas & Consumable Fuels  M3OG  10.487   9.813  11.625   
Materials Materials Materials                    M1M1  13.971  14.576  13.694   
...                                                       ...     ...     ...   
Utilities Utilities Utilities                    M2U2  16.033  15.189  16.982   
                    Electric Utilities           M3EU  16.315  15.333  17.118   
                    Gas Utilities                M3GU  16.938  15.918   17.94   
                    Multi-Utilities              M3MU   14.41  13.819   15.55   
                    Water Utilities              M3WU  17.276  18.123  19.263   

                                                              Earnings Growth  \
                                                                  Fiscal Year   
                                                          18M             FY0   
Energy    Energy    Energy                       M1E1  10.801         -85.188   
                                               

We can select any section of our data-frame for focused analysies

In [17]:
df["PEG-Ratio"]

Fiscal Year             \
                                                              FY0        FY1   
Energy    Energy    Energy                       M1E1    -0.92972  0.0227661   
                                                 M2E2    -0.92972  0.0227661   
                    Energy Equipment & Services  M3ES         NaN        NaN   
                    Oil, Gas & Consumable Fuels  M3OG   -0.923949  0.0227397   
Materials Materials Materials                    M1M1     3.64061   0.190147   
...                                                           ...        ...   
Utilities Utilities Utilities                    M2U2     4.17479    3.50375   
                    Electric Utilities           M3EU     2.21397   -13.1473   
                    Gas Utilities                M3GU     5.62296    2.42977   
                    Multi-Utilities              M3MU     50.2119   0.723416   
                    Water Utilities              M3WU   -0.874396    2.85425   

                                                                           
                                                            FY2       FY3  
Energy    Energy    Energy                       M1E1  0.613659   1.42127  
                                                 M2E2  0.613659   1.42127  
                    Energy Equipment & Services  M3ES  0.946008  0.746922  
                    Oil, Gas & Consumable Fuels  M3OG  0.612094   1.42755  
Materials Materials Materials                    M1M1  -3.46761   -3.3655  
...                                                         ...       ...  
Utilities Utilities Utilities                    M2U2   2.03181   2.73233  
                    Electric Utilities           M3EU   2.43762   2.39541  
                    Gas Utilities                M3GU   2.15743   2.48408  
                    Multi-Utilities              M3MU   1.36886    3.2363  
                    Water Utilities              M3WU     1.363  -3.87741  

[90 rows x 4 columns]

## Create the Python function ```Get_IBES_GA``` with [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html) [Dropdowns](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#Dropdown)

In [85]:
# Create a list to append with our returned data-frame:
DSWS_IBES_GA = []

# Define our drop down specificities:
drop_down_IBES_GA_return = widgets.Dropdown(
    options = [""] + xl_regions["Region"].to_list(),
    value = "",
    disabled = False)

# Define our drop down specificities for predefined variables:
drop_down_predefined_variables = widgets.Dropdown(
    options = [""] + ["Yes", "No"],
    value = "",
    description = "Use predefined ordered_mnemonic, IBESGA_fields, xl_index and xl_columns2 variables?",
    disabled = False)

In [86]:
# Create the function to programatically return the data-frame of interest
def Get_IBES_GA(area, # ' area ' example: "EAFE + Canada"
                loading_bar = True,
                display_df = True,
                append_DSWS_IBES_GA = True,
                ordered_mnemonic = "predefined",
                IBESGA_fields = "predefined",
                xl_index = "predefined",
                xl_columns2 = "predefined",
                max_row_col = True,
                export_to_excel = True):
    """Get_IBES_GA Version 1.0:
    If ' display_df ' is set to True, this function returns a dataframe of Datastream Web Service (DSWS) data of  Institutional Brokers' Estimate System (IBES) Global Aggregate earnings for country and regional sectors in an interactive way.
    If ' append_DSWS_IBES_GA ' is set to True, user need to have deffined a Python list ' DSWS_IBES_GA ', it will then be appended.
    DSWS is Refinitiv's API retreaving data from Datastream to a Python Pandas data-frame. For information on DSWS, please visit 'https://developers.refinitiv.com/en/api-catalog/eikon/datastream-web-service'.
    
    
    Parameters:
    ----------
    
    area: str
        Region of choice's name (e.g.: 'EAFE-ex-UK' or 'EAFE + Canada').
        It has to be one of the elements in the following list: ['EAFE' 'EAFE + Canada' 'EAFE-ex-UK' 'EASEA (EAFE-ex-Japan)'  'EM (Emerging Markets)' 'EM Asia' 'EM Eastern Europe' 'EM Europe'  'EM Europe + Middle East' 'EM Europe, Middle East & Africa' 'EM Far East'  'EM Latin America' 'EMU (Euro)' 'EMU (US Dollar)' 'EMU + UK' 'Europe'  'Europe-ex-EMU' 'Europe-ex-UK' 'Far East' 'G7 Index'  'Kokusai (World-ex-Japan)' 'Nordic Countries' 'North America' 'Pacific'  'Pacific-ex-Japan' 'World' 'World-ex-Australia' 'World-ex-EMU'  'World-ex-Europe' 'World-ex-UK' 'World-ex-USA' 'Argentina' 'Australia'  'Australia' 'Austria' 'Belgium' 'Brazil' 'Canada' 'Chile' 'China'  'Colombia' 'Czech Republic' 'Denmark' 'Egypt' 'Finland' 'France'  'Germany' 'Greece' 'Hong Kong' 'Hungary' 'India' 'Indonesia' 'Ireland'  'Israel' 'Italy' 'Japan' 'Jordan' 'Korea' 'Malaysia' 'Mexico' 'Morocco'  'Netherlands' 'New Zealand' 'Norway' 'Pakistan' 'Peru' 'Philippines'  'Poland' 'Portugal' 'Russia' 'Singapore' 'South Africa' 'Spain'  'Sri Lanka' 'Sweden' 'Switzerland' 'Taiwan' 'Thailand' 'Turkey'  'United Kingdom' 'USA' 'Venezuela']
        It has no default valuse, but as per the Example bellow, you may want to use this function in conjuncture with ' ipywidgets.interact '.
    
    loading_bar: Boolean
        If set to True, then a loading bar will aprear, keeping track of the dsws requests made - the steps that take longest in this function.
        Default: loading_bar = True
    
    display_df: Boolean
        If set to True, the resulting data-frame will be displayed.
        Default: display_df = True
    
    append_DSWS_IBES_GA: Boolean
        If set to True, user needs to have pre-created an empty Python lisy named ' DSWS_IBES_GA ' which will be populated with the data-frame returned.
        Default: append_DSWS_IBES_GA = True
    
    ordered_mnemonic: str or Pandas data-frame
        If set to "predefined", then a predefined Pandas data-frame of mnemonics is used.
        User may enter his/her own Pandas data-frame, but (s)he will need to change all of the following parameters accordingly: ordered_mnemonic, IBESGA_fields, xl_index, and xl_columns2.
        Default: ordered_mnemonic = "predefined"
    
    IBESGA_fields: str or Pandas data-frame
        If set to "predefined", then a predefined Pandas data-frame of Datastream IBESGA fields is used.
        User may enter his/her own Pandas data-frame, but (s)he will need to change all of the following parameters accordingly: ordered_mnemonic, IBESGA_fields, xl_index, and xl_columns2.
        Default: IBESGA_fields = "predefined"
    
    xl_index: str or Pandas data-frame
        If set to "predefined", then a predefined Pandas data-frame is used.
        User may enter his/her own Pandas data-frame, but (s)he will need to change all of the following parameters accordingly: ordered_mnemonic, IBESGA_fields, xl_index, and xl_columns2.
        Default: xl_index = "predefined"
    
    xl_columns2: str or Pandas data-frame
        If set to "predefined", then a predefined Pandas data-frame is used.
        User may enter his/her own Pandas data-frame, but (s)he will need to change all of the following parameters accordingly: ordered_mnemonic, IBESGA_fields, xl_index, and xl_columns2.
        Default: xl_columns2 = "predefined"
    
    max_row_col: Boolean
        If set to True, all of the resulted data-frame's columns and indecies (rows) will be displayed.
        Default: max_row_col = True
    
    export_to_excel: Boolean
        If se to True, an excel workbook with one sheet of the returned data-frame will be generated where the python file is run.
        Default: True
    
    
    Dependencies:
    ----------
    
    pandas 1.1.4 as pd
    numpy 1.20.1 as np
    tqdm 4.48.2 via ' from tqdm.notebook import trange '
    DatastreamDSWS as dsws
    warnings
    
    Optional:
    
    ipywidgets 7.5.1
    
    
    Examples:
    ----------
    
    >>> import DatastreamDSWS as dsws
    >>> ds = dsws.Datastream(username = "insert dsws username here", password = "insert dsws password here")
    >>> from datetime import datetime
    >>> import warnings
    >>> from datetime import date
    >>> import pandas as pd
    >>> import numpy as np
    >>> import ipywidgets as widgets
    >>> from IPython.display import display
    >>> 
    >>> xl_regions = pd.DataFrame(data = {'Region': {0: 'EAFE', 1: 'EAFE + Canada',2: 'EAFE-ex-UK',3: 'EASEA (EAFE-ex-Japan)',4: 'EM (Emerging Markets)',5: 'EM Asia',6: 'EM Eastern Europe',7: 'EM Europe',8: 'EM Europe + Middle East',9: 'EM Europe, Middle East & Africa',10: 'EM Far East',11: 'EM Latin America',12: 'EMU (Euro)',13: 'EMU (US Dollar)',14: 'EMU + UK',15: 'Europe',16: 'Europe-ex-EMU',17: 'Europe-ex-UK',18: 'Far East',19: 'G7 Index',20: 'Kokusai (World-ex-Japan)',21: 'Nordic Countries',22: 'North America',23: 'Pacific',24: 'Pacific-ex-Japan',25: 'World',26: 'World-ex-Australia',27: 'World-ex-EMU',28: 'World-ex-Europe',29: 'World-ex-UK',30: 'World-ex-USA'},'Mnemonic': {0: 'EA',1: 'FC',2: 'EX',3: 'AS',4: 'EF',5: 'MI',6: 'EB',7: 'ME',8: 'UE',9: 'EM',10: 'MP',11: 'ML',12: 'MU',13: 'M$',14: 'MK',15: 'UR',16: 'EE',17: 'UX',18: 'FE',19: 'G7',20: 'KK',21: 'NC',22: 'NA',23: 'PC',24: 'PX',25: 'DW',26: 'WA',27: 'WE',28: 'WC',29: 'WK',30: 'WS'},'Code': {0: '@:M1EAFE',1: '@:M1EAFEC',2: '@:M1EAFEU',3: '@:M1EASEA',4: '@:M2EMG',5: '@:M2ASIA',6: '@:M2EEUR',7: '@:M2EUR',8: '@:M2EURME',9: '@:M2EMEA',10: '@:M2FARE',11: '@:M2LATAM',12: '@:MUMSCI',13: '@:M1EMU',14: '@:MKMSCI',15: '@:M1EROP',16: '@:EEMSCI',17: '@:M1EURXU',18: '@:M1FARE',19: '@:G7MSCI',20: '@:M1KOKUS',21: '@:M1NORD',22: '@:M1NAMER',23: '@:M1PAC',24: '@:M1PACXJ',25: '@:M1WRLD',26: '@:WAMSCI',27: '@:WEMSCI',28: '@:WCMSCI',29: '@:M1WLDXU',30: '@:M1WLDXA'}})
    >>> xl_countries = pd.DataFrame(data = {'Region': {0: 'Argentina',1: 'Australia',2: 'Australia',3: 'Austria',4: 'Belgium',5: 'Brazil',6: 'Canada',7: 'Chile',8: 'China',9: 'Colombia',10: 'Czech Republic',11: 'Denmark',12: 'Egypt',13: 'Finland',14: 'France',15: 'Germany',16: 'Greece',17: 'Hong Kong',18: 'Hungary',19: 'India',20: 'Indonesia',21: 'Ireland',22: 'Israel',23: 'Italy',24: 'Japan',25: 'Jordan',26: 'Korea',27: 'Malaysia',28: 'Mexico',29: 'Morocco',30: 'Netherlands',31: 'New Zealand',32: 'Norway',33: 'Pakistan',34: 'Peru',35: 'Philippines',36: 'Poland',37: 'Portugal',38: 'Russia',39: 'Singapore',40: 'South Africa',41: 'Spain',42: 'Sri Lanka',43: 'Sweden',44: 'Switzerland',45: 'Taiwan',46: 'Thailand',47: 'Turkey',48: 'United Kingdom',49: 'USA',50: 'Venezuela'},'Mnemonic': {0: 'AR',1: 'AU',2: 'AU',3: 'AT',4: 'BE',5: 'BR',6: 'CA',7: 'CL',8: 'CN',9: 'CO',10: 'CZ',11: 'DK',12: 'EG',13: 'FI',14: 'FR',15: 'DE',16: 'GR',17: 'HK',18: 'HU',19: 'IN',20: 'ID',21: 'IE',22: 'IL',23: 'IT',24: 'JP',25: 'JO',26: 'KR',27: 'MY',28: 'MX',29: 'MA',30: 'NL',31: 'NZ',32: 'NO',33: 'PK',34: 'PE',35: 'PH',36: 'PL',37: 'PT',38: 'RU',39: 'SG',40: 'SA',41: 'ES',42: 'LK',43: 'SE',44: 'SW',45: 'TW',46: 'TH',47: 'TR',48: 'UK',49: 'US',50: 'VE'}, 'Code': {0: '@:ARMSCIP',1: '@:MSCIP',2: '@:AUMSCIP',3: '@:OEMSCIP',4: '@:BGMSCIP',5: '@:BRMSCIP',6: '@:CNMSCIP',7: '@:CLMSCIP',8: '@:CHMSCIP',9: '@:CBMSCIP',10: '@:CZMSCIP',11: '@:DKMSCIP',12: '@:EYMSCIP',13: '@:FNMSCIP',14: '@:FRMSCIP',15: '@:BDMSCIP',16: '@:GRMSCIP',17: '@:HKMSCIP',18: '@:HNMSCIP',19: '@:INMSCIP',20: '@:IDMSCIP',21: '@:IRMSCIP',22: '@:ISMSCIP',23: '@:ITMSCIP',24: '@:JPMSCIP',25: '@:JOMSCIP',26: '@:KOMSCIP',27: '@:MYMSCIP',28: '@:MXMSCIP',29: '@:MCMSCIP',30: '@:NLMSCIP',31: '@:NZMSCIP',32: '@:NWMSCIP',33: '@:PKMSCIP',34: '@:PEMSCIP',35: '@:PHMSCIP',36: '@:POMSCIP',37: '@:PTMSCIP',38: '@:RSMSCIP',39: '@:SGMSCIP',40: '@:SAMSCIP',41: '@:ESMSCIP',42: '@:CYMSCIP',43: '@:SDMSCIP',44: '@:SWMSCIP',45: '@:TAMSCIP',46: '@:THMSCIP',47: '@:TKMSCIP',48: '@:UKMSCIP',49: '@:USMSCIP',50: '@:VEMSCIP'}})
    >>> xl_regions = xl_regions.append(xl_countries, ignore_index = True)
    >>> 
    >>> DSWS_IBES_GA = []
    >>> 
    >>> drop_down_IBES_GA_return = widgets.Dropdown(options = [""] + xl_regions["Region"].to_list(),value = "", disabled = False)
    >>> 
    >>> drop_down_predefined_variables = widgets.Dropdown(options = [""] + ["Yes", "No"], value = "", description = "Use predefined ordered_mnemonic, IBESGA_fields, xl_index and xl_columns2 variables?", disabled = False)
    >>> 
    >>> def Get_IBES_GA_predefined(area):
    >>>     _df = Get_IBES_GA(area)
    >>>     display(_df)
    >>> 
    >>> def IBES_GA_predefined_variables(predefined):
    >>>     if predefined == "Yes":
    >>>         widgets.interact(Get_IBES_GA_predefined, area = drop_down_IBES_GA_return);
    >>>     elif predefined == "No":
    >>>         widgets.interact(Get_IBES_GA, area = drop_down_IBES_GA_return);
    >>> 
    >>> widgets.interact(IBES_GA_predefined_variables, predefined = drop_down_predefined_variables);
    >>> 
    >>> display(DSWS_IBES_GA[0])
    """
    
    # If the ' area ' is chosen in the following dropdown as an empty value (i.e.: ""), then we don't want to return anything:
    if area == "":
        pass
    else:
        
        # Set data-frame display conditions:
        if max_row_col == True:
            pd.set_option('display.max_row', None)
            pd.set_option('display.max_columns', None)
        
        # # Define reference Python objects:
        
        if ordered_mnemonic == "predefined": ordered_mnemonic = "M1E1,M2E2,M3ES,M3OG,M1M1,M2M2,M3CH,M3CM,M3CT,M3MM,M3PF,M1ID,M2CG,M3AD,M3BP,M3CN,M3EI,M3IC,M3MC,M3TC,M2C2,M3C3,M2TR,M3AF,M3AL,M3MA,M3RR,M3TI,M1CD,M2AC,M3AU,M3AM,M2CA,M3HD,M3LE,M3TA,M2HR,M3HR,M2MD,M3ME,M2RT,M3DI,M3NT,M3MR,M3SR,M1CS,M2FD,M3FD,M2FB,M3BV,M3FP,M3TB,M2HH,M3HP,M3PP,M1HC,M2HE,M3HS,M3PS,M2PB,M3BI,M3PH,M1FN,M2B2,M3B3,M2D2,M3D3,M2I2,M3I3,M1RE,M1IT,M2SS,M3NS,M3IS,M3SW,M2TH,M3CE,M3CP,M3EE,M3SC,M1T1,M2T2,M3DT,M3WT,M1U1,M2U2,M3EU,M3GU,M3MU,M3WU"
        else: ordered_mnemonic = ordered_mnemonic
        
        # The long string for ' IBESGA_fields ' (and subsequently ' IBESGA_tickers_str ') can be found in the ' Datastream IBES Global Aggregates MSCI .xlsm ' file.
        if IBESGA_fields == "predefined":
            IBESGA_fields = "ALNAME,AF0PE,AF1PE,AF2PE,AF3PE,A12PE,A18PE,AF0GRO,AF1GRO,AF2GRO,AF3GRO,AF0VAL,AF1MN,AF2MN,AF3MN,A12FE,A18FE,1/X(AF0PE)*100-X(AGBYLD),1/X(AF1PE)*100-X(AGBYLD),1/X(AF2PE)*100-X(AGBYLD),1/X(AF3PE)*100-X(AGBYLD),1/X(A12PE)*100-X(AGBYLD),1/X(A18PE)*100-X(AGBYLD),X(AZ0VAL)/X(AT0VAL),X(AZ1MN)/X(AT1MN),X(AZ2MN)/X(AT2MN),X(AZ3MN)/X(AT3MN),X(AZ12FZ)/X(AT12FT),X(AZ18FZ)/X(AT18FT),429E*1.0000,ADVYLD"
            IBESGA_fields = IBESGA_fields.split(",")
            # Callin it with ' _str ' at the end to disosiate it from other objects and use it later.
            IBESGA_tickers_str = "M1CD,M1CS,M1E1,M1FN,M1HC,M1ID,M1IT,M1M1,M1T1,M1U1,M1RE,M2AC,M2B2,M2C2,M2CA,M2CG,M2D2,M2E2,M2FB,M2FD,M2HE,M2HH,M2HR,M2I2,M2M2,M2MD,M2PB,M2RT,M2SS,M2T2,M2TH,M2TR,M2U2,M3AD,M3AF,M3AL,M3AM,M3AU,M3B3,M3BI,M3BP,M3BV,M3C3,M3CE,M3CH,M3CM,M3CN,M3CP,M3CT,M3D3,M3DI,M3DT,M3EE,M3EI,M3ES,M3EU,M3FD,M3FP,M3GU,M3HD,M3HP,M3HR,M3HS,M3I3,M3IC,M3IS,M3LE,M3MA,M3MC,M3ME,M3MM,M3MR,M3MU,M3NS,M3NT,M3OG,M3PF,M3PH,M3PP,M3PS,M3RR,M3SC,M3SR,M3SW,M3TA,M3TB,M3TC,M3TI,M3WT,M3WU"
            
            
            # # Define the list of tickers to pull data for from DSWS
            
            def Region_to_DSWS_region_mnemonic_and_IBES_code(region = None):
                """For description of this function, see 'How to collect Datastream IBES Global Aggregate Earnings Data' article on the Refinitiv/LSEG Developer Comunity Article Calalogue."""
                xl_regions = pd.DataFrame(data = {'Region': {0: 'EAFE', 1: 'EAFE + Canada', 2: 'EAFE-ex-UK', 3: 'EASEA (EAFE-ex-Japan)', 4: 'EM (Emerging Markets)', 5: 'EM Asia', 6: 'EM Eastern Europe', 7: 'EM Europe', 8: 'EM Europe + Middle East', 9: 'EM Europe, Middle East & Africa', 10: 'EM Far East', 11: 'EM Latin America', 12: 'EMU (Euro)', 13: 'EMU (US Dollar)', 14: 'EMU + UK', 15: 'Europe', 16: 'Europe-ex-EMU', 17: 'Europe-ex-UK', 18: 'Far East', 19: 'G7 Index', 20: 'Kokusai (World-ex-Japan)', 21: 'Nordic Countries', 22: 'North America', 23: 'Pacific', 24: 'Pacific-ex-Japan', 25: 'World', 26: 'World-ex-Australia', 27: 'World-ex-EMU', 28: 'World-ex-Europe', 29: 'World-ex-UK', 30: 'World-ex-USA', 31: 'Argentina', 32: 'Australia', 33: 'Australia', 34: 'Austria', 35: 'Belgium', 36: 'Brazil', 37: 'Canada', 38: 'Chile', 39: 'China', 40: 'Colombia', 41: 'Czech Republic', 42: 'Denmark', 43: 'Egypt', 44: 'Finland', 45: 'France', 46: 'Germany', 47: 'Greece', 48: 'Hong Kong', 49: 'Hungary', 50: 'India', 51: 'Indonesia', 52: 'Ireland', 53: 'Israel', 54: 'Italy', 55: 'Japan', 56: 'Jordan', 57: 'Korea', 58: 'Malaysia', 59: 'Mexico', 60: 'Morocco', 61: 'Netherlands', 62: 'New Zealand', 63: 'Norway', 64: 'Pakistan', 65: 'Peru', 66: 'Philippines', 67: 'Poland', 68: 'Portugal', 69: 'Russia', 70: 'Singapore', 71: 'South Africa', 72: 'Spain', 73: 'Sri Lanka', 74: 'Sweden', 75: 'Switzerland', 76: 'Taiwan', 77: 'Thailand', 78: 'Turkey', 79: 'United Kingdom', 80: 'USA', 81: 'Venezuela'}, 'Mnemonic': {0: 'EA', 1: 'FC', 2: 'EX', 3: 'AS', 4: 'EF', 5: 'MI', 6: 'EB', 7: 'ME', 8: 'UE', 9: 'EM', 10: 'MP', 11: 'ML', 12: 'MU', 13: 'M$', 14: 'MK', 15: 'UR', 16: 'EE', 17: 'UX', 18: 'FE', 19: 'G7', 20: 'KK', 21: 'NC', 22: 'NA', 23: 'PC', 24: 'PX', 25: 'DW', 26: 'WA', 27: 'WE', 28: 'WC', 29: 'WK', 30: 'WS', 31: 'AR', 32: 'AU', 33: 'AU', 34: 'AT', 35: 'BE', 36: 'BR', 37: 'CA', 38: 'CL', 39: 'CN', 40: 'CO', 41: 'CZ', 42: 'DK', 43: 'EG', 44: 'FI', 45: 'FR', 46: 'DE', 47: 'GR', 48: 'HK', 49: 'HU', 50: 'IN', 51: 'ID', 52: 'IE', 53: 'IL', 54: 'IT', 55: 'JP', 56: 'JO', 57: 'KR', 58: 'MY', 59: 'MX', 60: 'MA', 61: 'NL', 62: 'NZ', 63: 'NO', 64: 'PK', 65: 'PE', 66: 'PH', 67: 'PL', 68: 'PT', 69: 'RU', 70: 'SG', 71: 'SA', 72: 'ES', 73: 'LK', 74: 'SE', 75: 'SW', 76: 'TW', 77: 'TH', 78: 'TR', 79: 'UK', 80: 'US', 81: 'VE'}, 'Code': {0: '@:M1EAFE', 1: '@:M1EAFEC', 2: '@:M1EAFEU', 3: '@:M1EASEA', 4: '@:M2EMG', 5: '@:M2ASIA', 6: '@:M2EEUR', 7: '@:M2EUR', 8: '@:M2EURME', 9: '@:M2EMEA', 10: '@:M2FARE', 11: '@:M2LATAM', 12: '@:MUMSCI', 13: '@:M1EMU', 14: '@:MKMSCI', 15: '@:M1EROP', 16: '@:EEMSCI', 17: '@:M1EURXU', 18: '@:M1FARE', 19: '@:G7MSCI', 20: '@:M1KOKUS', 21: '@:M1NORD', 22: '@:M1NAMER', 23: '@:M1PAC', 24: '@:M1PACXJ', 25: '@:M1WRLD', 26: '@:WAMSCI', 27: '@:WEMSCI', 28: '@:WCMSCI', 29: '@:M1WLDXU', 30: '@:M1WLDXA', 31: '@:ARMSCIP', 32: '@:MSCIP', 33: '@:AUMSCIP', 34: '@:OEMSCIP', 35: '@:BGMSCIP', 36: '@:BRMSCIP', 37: '@:CNMSCIP', 38: '@:CLMSCIP', 39: '@:CHMSCIP', 40: '@:CBMSCIP', 41: '@:CZMSCIP', 42: '@:DKMSCIP', 43: '@:EYMSCIP', 44: '@:FNMSCIP', 45: '@:FRMSCIP', 46: '@:BDMSCIP', 47: '@:GRMSCIP', 48: '@:HKMSCIP', 49: '@:HNMSCIP', 50: '@:INMSCIP', 51: '@:IDMSCIP', 52: '@:IRMSCIP', 53: '@:ISMSCIP', 54: '@:ITMSCIP', 55: '@:JPMSCIP', 56: '@:JOMSCIP', 57: '@:KOMSCIP', 58: '@:MYMSCIP', 59: '@:MXMSCIP', 60: '@:MCMSCIP', 61: '@:NLMSCIP', 62: '@:NZMSCIP', 63: '@:NWMSCIP', 64: '@:PKMSCIP', 65: '@:PEMSCIP', 66: '@:PHMSCIP', 67: '@:POMSCIP', 68: '@:PTMSCIP', 69: '@:RSMSCIP', 70: '@:SGMSCIP', 71: '@:SAMSCIP', 72: '@:ESMSCIP', 73: '@:CYMSCIP', 74: '@:SDMSCIP', 75: '@:SWMSCIP', 76: '@:TAMSCIP', 77: '@:THMSCIP', 78: '@:TKMSCIP', 79: '@:UKMSCIP', 80: '@:USMSCIP', 81: '@:VEMSCIP'}})
                if region not in xl_regions["Region"].to_list():
                    warnings.warn("Invalid ' region ' argument specified.")
                else:
                    return xl_regions[xl_regions["Region"] == region]["Mnemonic"].values[0], xl_regions[xl_regions["Region"] == region]["Code"].values[0]
            
            IBESGA_full_tickers = ["@:" + Region_to_DSWS_region_mnemonic_and_IBES_code(area)[0] + i for i in IBESGA_tickers_str.split(",")]
        
        else:
            IBESGA_full_tickers = IBESGA_fields
        
        # # Get coding:
        
        # Create 1st data-frame to subsequentally append
        df0 = ds.get_data(tickers = IBESGA_full_tickers[0],
                          fields = IBESGA_fields[:15],
                          kind = 0)
        df1 = df0.append(ds.get_data(tickers = IBESGA_full_tickers[0],
                                     fields = IBESGA_fields[15:],
                                     kind = 0),
                         ignore_index = True)
        
        ## Append our data-frame with each ticker
        
        # Leave the option for a loading bar:
        if loading_bar == True: J = trange(len(IBESGA_full_tickers), # colour = '#001EFF',
                                                         desc = 'requests', write_bytes = True)
        else: J = range(len(IBESGA_full_tickers[1:]))
        
        # Now request data:
        for i,j in zip(IBESGA_full_tickers, J):
            
            if i == IBESGA_full_tickers[0]:
                pass
            else:            
                # Create a placeholder data-frame ' _df ' to append onto our previously defined data-frame ' df1 '
                _df1 = ds.get_data(tickers = i,
                                  fields = IBESGA_fields[:15],
                                  kind = 0)
                _df2 = ds.get_data(tickers = i,
                                  fields = IBESGA_fields[15:],
                                  kind = 0)
                _df = _df1.append(_df2, ignore_index = True)
                df1 = df1.append(_df, ignore_index = True)
        
        # Rearange our data-frame to have tickers as index and fields as columns
        df2 = pd.DataFrame(
            index = df1["Instrument"].unique(), columns = df1["Datatype"].unique(),
            data = [list(df1["Value"][df1["Instrument"] == i]) for i in df1["Instrument"].unique()])
        # Replace str 'NA' with numpy nan values
        df2 = df2.replace(['NA'], np.nan)
        for i in range(4):
            df2[f"PEGRatio{i}"] = df2[f"AF{i}PE"] / df2[f"AF{i}GRO"]
        # Need to rearange the columns in order
        df2 = df2.reindex(df2.columns[1:11].tolist() + [f"PEGRatio{i}" for i in range(4)] + df2.columns[11:-4].tolist(), axis=1)
        # We don't want to loose the data of which row is for which ticker with the next few lines, so let's create a column with that information
        df2["Tickers"] = df2.index
        # Rename our data-frame's index to fit a standard easier to work with
        df2.index = IBESGA_tickers_str.split(",")
        # Rearange the order of the rows
        df3 = df2.T[ordered_mnemonic.split(",")].T
        
        # We will use the following ' xl_index ' data-frame to index our returned data-frame
        if xl_index == "predefined":
            xl_index = pd.DataFrame(data = {'Category': {0: 'Energy', 1: 'Energy', 2: 'Energy', 3: 'Energy', 4: 'Materials', 5: 'Materials', 6: 'Materials', 7: 'Materials', 8: 'Materials', 9: 'Materials', 10: 'Materials', 11: 'Industrials', 12: 'Industrials', 13: 'Industrials', 14: 'Industrials', 15: 'Industrials', 16: 'Industrials', 17: 'Industrials', 18: 'Industrials', 19: 'Industrials', 20: 'Industrials', 21: 'Industrials', 22: 'Industrials', 23: 'Industrials', 24: 'Industrials', 25: 'Industrials', 26: 'Industrials', 27: 'Industrials', 28: 'Consumer Discretionary', 29: 'Consumer Discretionary', 30: 'Consumer Discretionary', 31: 'Consumer Discretionary', 32: 'Consumer Discretionary', 33: 'Consumer Discretionary', 34: 'Consumer Discretionary', 35: 'Consumer Discretionary', 36: 'Consumer Discretionary', 37: 'Consumer Discretionary', 38: 'Consumer Discretionary', 39: 'Consumer Discretionary', 40: 'Consumer Discretionary', 41: 'Consumer Discretionary', 42: 'Consumer Discretionary', 43: 'Consumer Discretionary', 44: 'Consumer Discretionary', 45: 'Consumer Staples', 46: 'Consumer Staples', 47: 'Consumer Staples', 48: 'Consumer Staples', 49: 'Consumer Staples', 50: 'Consumer Staples', 51: 'Consumer Staples', 52: 'Consumer Staples', 53: 'Consumer Staples', 54: 'Consumer Staples', 55: 'Health Care', 56: 'Health Care', 57: 'Health Care', 58: 'Health Care', 59: 'Health Care', 60: 'Health Care', 61: 'Health Care', 62: 'Financials', 63: 'Financials', 64: 'Financials', 65: 'Financials', 66: 'Financials', 67: 'Financials', 68: 'Financials', 69: 'Real Estate', 70: 'Information Technology', 71: 'Information Technology', 72: 'Information Technology', 73: 'Information Technology', 74: 'Information Technology', 75: 'Information Technology', 76: 'Information Technology', 77: 'Information Technology', 78: 'Information Technology', 79: 'Information Technology', 80: 'Telecommunication Services', 81: 'Telecommunication Services', 82: 'Telecommunication Services', 83: 'Telecommunication Services', 84: 'Utilities', 85: 'Utilities', 86: 'Utilities', 87: 'Utilities', 88: 'Utilities', 89: 'Utilities'}, 'Sub-Category': {0: 'Energy', 1: 'Energy', 2: 'Energy', 3: 'Energy', 4: 'Materials', 5: 'Materials', 6: 'Materials', 7: 'Materials', 8: 'Materials', 9: 'Materials', 10: 'Materials', 11: 'Industrials', 12: 'Capital Goods', 13: 'Capital Goods', 14: 'Capital Goods', 15: 'Capital Goods', 16: 'Capital Goods', 17: 'Capital Goods', 18: 'Capital Goods', 19: 'Capital Goods', 20: 'Commercial Services & Supplies', 21: 'Commercial Services & Supplies', 22: 'Transportation', 23: 'Transportation', 24: 'Transportation', 25: 'Transportation', 26: 'Transportation', 27: 'Transportation', 28: 'Consumer Discretionary', 29: 'Automobiles & Components', 30: 'Automobiles & Components', 31: 'Automobiles & Components', 32: 'Consumer Durables & Apparel', 33: 'Consumer Durables & Apparel', 34: 'Consumer Durables & Apparel', 35: 'Consumer Durables & Apparel', 36: 'Hotels, Restaurants & Leisure', 37: 'Hotels, Restaurants & Leisure', 38: 'Media', 39: 'Media', 40: 'Retailing', 41: 'Retailing', 42: 'Retailing', 43: 'Retailing', 44: 'Retailing', 45: 'Consumer Staples', 46: 'Food & Staples Retailing', 47: 'Food & Staples Retailing', 48: 'Food Beverage & Tobacco', 49: 'Food Beverage & Tobacco', 50: 'Food Beverage & Tobacco', 51: 'Food Beverage & Tobacco', 52: 'Household & Personal Products', 53: 'Household & Personal Products', 54: 'Household & Personal Products', 55: 'Health Care', 56: 'Health Care Equipment & Services', 57: 'Health Care Equipment & Services', 58: 'Health Care Equipment & Services', 59: 'Pharmaceuticals, Biotechnology & Life Sciences', 60: 'Pharmaceuticals, Biotechnology & Life Sciences', 61: 'Pharmaceuticals, Biotechnology & Life Sciences', 62: 'Financials', 63: 'Banks', 64: 'Banks', 65: 'Diversified Financials', 66: 'Diversified Financials', 67: 'Insurance', 68: 'Insurance', 69: 'Real Estate', 70: 'Information Technology', 71: 'Software & Services', 72: 'Software & Services', 73: 'Software & Services', 74: 'Software & Services', 75: 'Technology Hardware & Equipment', 76: 'Technology Hardware & Equipment', 77: 'Technology Hardware & Equipment', 78: 'Technology Hardware & Equipment', 79: 'Technology Hardware & Equipment', 80: 'Telecommunication Services', 81: 'Telecommunication Services', 82: 'Telecommunication Services', 83: 'Telecommunication Services', 84: 'Utilities', 85: 'Utilities', 86: 'Utilities', 87: 'Utilities', 88: 'Utilities', 89: 'Utilities'}, 'Sector': {0: 'Energy', 1: 'Energy', 2: 'Energy Equipment & Services ', 3: 'Oil, Gas & Consumable Fuels ', 4: 'Materials', 5: 'Materials', 6: 'Chemicals ', 7: 'Construction Materials ', 8: 'Containers & Packaging ', 9: 'Metals & Mining ', 10: 'Paper & Forest Products ', 11: 'Industrials', 12: 'Capital Goods', 13: 'Aerospace & Defense ', 14: 'Building Products ', 15: 'Construction & Engineering ', 16: 'Electronic Equipment & Instruments', 17: 'Industrial Conglomerates ', 18: 'Machinery ', 19: 'Trading Companies & Distributors Industry', 20: 'Commercial Services & Supplies', 21: 'Commercial Services & Supplies ', 22: 'Transportation', 23: 'Air Freight & Couriers ', 24: 'Airlines ', 25: 'Marine ', 26: 'Road & Rail ', 27: 'Transportation Infrastructure ', 28: 'Consumer Discretionary', 29: 'Automobiles & Components', 30: 'Auto Components ', 31: 'Automobiles ', 32: 'Consumer Durables & Apparel', 33: 'Household Durables ', 34: 'Leisure Equipment & Products ', 35: 'Textiles & Apparel ', 36: 'Hotels, Restaurants & Leisure', 37: 'Hotels Restaurants & Leisure ', 38: 'Media', 39: 'Media ', 40: 'Retailing', 41: 'Distributors ', 42: 'Internet & Catalog Retail ', 43: 'Multiline Retail ', 44: 'Specialty Retail ', 45: 'Consumer Staples', 46: 'Food & Staples Retailing', 47: 'Food & Staples Retailing ', 48: 'Food Beverage & Tobacco', 49: 'Beverages ', 50: 'Food Products ', 51: 'Tobacco ', 52: 'Household & Personal Products', 53: 'Household Products ', 54: 'Personal Products ', 55: 'Health Care', 56: 'Health Care Equipment & Services', 57: 'Health Care Equipment & Supplies', 58: 'Health Care Providers & Services', 59: 'Pharmaceuticals, Biotechnology & Life Sciences', 60: 'Biotechnology ', 61: 'Pharmaceuticals ', 62: 'Financials', 63: 'Banks', 64: 'Banks', 65: 'Diversified Financials', 66: 'Diversified Financial Services ', 67: 'Insurance', 68: 'Insurance ', 69: 'Real Estate', 70: 'Information Technology', 71: 'Software & Services', 72: 'Internet Software & Services ', 73: 'IT Consulting & Services ', 74: 'Software ', 75: 'Technology Hardware & Equipment', 76: 'Communications Equipment ', 77: 'Computers & Peripherals ', 78: 'Electrical Equipment ', 79: 'Semiconductors & Semiconductor Equipment ', 80: 'Telecommunication Services', 81: 'Telecommunication Services', 82: 'Diversified Telecommunications Services   ', 83: 'Wireless Telecommunication Services', 84: 'Utilities', 85: 'Utilities', 86: 'Electric Utilities ', 87: 'Gas Utilities ', 88: 'Multi-Utilities ', 89: 'Water Utilities '}, 'Mnemonic': {0: 'M1E1', 1: 'M2E2', 2: 'M3ES', 3: 'M3OG', 4: 'M1M1', 5: 'M2M2', 6: 'M3CH', 7: 'M3CM', 8: 'M3CT', 9: 'M3MM', 10: 'M3PF', 11: 'M1ID', 12: 'M2CG', 13: 'M3AD', 14: 'M3BP', 15: 'M3CN', 16: 'M3EI', 17: 'M3IC', 18: 'M3MC', 19: 'M3TC', 20: 'M2C2', 21: 'M3C3', 22: 'M2TR', 23: 'M3AF', 24: 'M3AL', 25: 'M3MA', 26: 'M3RR', 27: 'M3TI', 28: 'M1CD', 29: 'M2AC', 30: 'M3AU', 31: 'M3AM', 32: 'M2CA', 33: 'M3HD', 34: 'M3LE', 35: 'M3TA', 36: 'M2HR', 37: 'M3HR', 38: 'M2MD', 39: 'M3ME', 40: 'M2RT', 41: 'M3DI', 42: 'M3NT', 43: 'M3MR', 44: 'M3SR', 45: 'M1CS', 46: 'M2FD', 47: 'M3FD', 48: 'M2FB', 49: 'M3BV', 50: 'M3FP', 51: 'M3TB', 52: 'M2HH', 53: 'M3HP', 54: 'M3PP', 55: 'M1HC', 56: 'M2HE', 57: 'M3HS', 58: 'M3PS', 59: 'M2PB', 60: 'M3BI', 61: 'M3PH', 62: 'M1FN', 63: 'M2B2', 64: 'M3B3', 65: 'M2D2', 66: 'M3D3', 67: 'M2I2', 68: 'M3I3', 69: 'M1RE', 70: 'M1IT', 71: 'M2SS', 72: 'M3NS', 73: 'M3IS', 74: 'M3SW', 75: 'M2TH', 76: 'M3CE', 77: 'M3CP', 78: 'M3EE', 79: 'M3SC', 80: 'M1T1', 81: 'M2T2', 82: 'M3DT', 83: 'M3WT', 84: 'M1U1', 85: 'M2U2', 86: 'M3EU', 87: 'M3GU', 88: 'M3MU', 89: 'M3WU'}})
        else:
            xl_index = xl_index
        
        # Now re-index the data-frame:
        _columns = [list(xl_index.fillna(method='ffill').loc[i].values) for i in range(len(xl_index.index))]
        df3.index = pd.MultiIndex.from_tuples(_columns)
        
        ## Now we need to rename the columns:
        
        # If you have the acompanying excel workbook, you can run: ' xl_columns2 = pd.read_excel("Reference data.xls", sheet_name = "Column Names 2").fillna(method='ffill') ''
        if xl_columns2 == "predefined":
            xl_columns2 = pd.DataFrame(data = {'Column 1': {0: 'PE-Ratio', 1: 'PE-Ratio', 2: 'PE-Ratio', 3: 'PE-Ratio', 4: 'PE-Ratio', 5: 'PE-Ratio', 6: 'Earnings Growth', 7: 'Earnings Growth', 8: 'Earnings Growth', 9: 'Earnings Growth', 10: 'PEG-Ratio', 11: 'PEG-Ratio', 12: 'PEG-Ratio', 13: 'PEG-Ratio', 14: 'EPS Mean', 15: 'EPS Mean', 16: 'EPS Mean', 17: 'EPS Mean', 18: 'EPS Mean', 19: 'EPS Mean', 20: 'Risk premium', 21: 'Risk premium', 22: 'Risk premium', 23: 'Risk premium', 24: 'Risk premium', 25: 'Risk premium', 26: 'EV/EBITDA', 27: 'EV/EBITDA', 28: 'EV/EBITDA', 29: 'EV/EBITDA', 30: 'EV/EBITDA', 31: 'EV/EBITDA', 32: 'Rev. Ratio', 33: 'Div Yield', 34: 'Tickers'}, 'Column 2': {0: 'Fiscal Year', 1: 'Fiscal Year', 2: 'Fiscal Year', 3: 'Fiscal Year', 4: 'Forward', 5: 'Forward', 6: 'Fiscal Year', 7: 'Fiscal Year', 8: 'Fiscal Year', 9: 'Fiscal Year', 10: 'Fiscal Year', 11: 'Fiscal Year', 12: 'Fiscal Year', 13: 'Fiscal Year', 14: 'Fiscal Year', 15: 'Fiscal Year', 16: 'Fiscal Year', 17: 'Fiscal Year', 18: 'Forward', 19: 'Forward', 20: 'Fiscal Year', 21: 'Fiscal Year', 22: 'Fiscal Year', 23: 'Fiscal Year', 24: 'Forward', 25: 'Forward', 26: 'Fiscal Year', 27: 'Fiscal Year', 28: 'Fiscal Year', 29: 'Fiscal Year', 30: 'Forward', 31: 'Forward', 32: 'Rev. Ratio', 33: 'Div Yield', 34: 'Tickers'}, 'Column 3': {0: 'FY0', 1: 'FY1', 2: 'FY2', 3: 'FY3', 4: '12M', 5: '18M', 6: 'FY0', 7: 'FY1', 8: 'FY2', 9: 'FY3', 10: 'FY0', 11: 'FY1', 12: 'FY2', 13: 'FY3', 14: 'FY0', 15: 'FY1', 16: 'FY2', 17: 'FY3', 18: '12M', 19: '18M', 20: 'FY0', 21: 'FY1', 22: 'FY2', 23: 'FY3', 24: '12M', 25: '18M', 26: 'FY0', 27: 'FY1', 28: 'FY2', 29: 'FY3', 30: '12M', 31: '18M', 32: 'Rev. Ratio', 33: 'Div Yield', 34: 'Tickers'}})
        else:
            xl_columns2 = xl_columns2
        
        _columns = [list(xl_columns2.loc[i].values) for i in range(len(xl_columns2.index))]
        df3.columns = pd.MultiIndex.from_tuples(_columns)
        
        # Append out previously defined list with the dataframe created
        try:
            if append_DSWS_IBES_GA == True: DSWS_IBES_GA.append(df3)
        except:
            warnings.warn("If ' append_DSWS_IBES_GA ' is set to True, user needs to define an empty Python list ' append_DSWS_IBES_GA ' before running ' Get_IBES_GA '.")
        
        # Export to excel sheet if asked:
        if export_to_excel == True:
            df3.to_excel(excel_writer = f"DSWS_IBES_GA_{area}.xlsx",
                         sheet_name = datetime.now().strftime("%Y.%m.%d_%Hh.%Mm"))
        
        # Just to check if this function worked expectedly, let's return the number of rows in the outputed data-frame
        if display_df == True: return df3

Let's setup and use an interactive widget:

In [87]:
def Get_IBES_GA_predefined(area):
    _df = Get_IBES_GA(area)
    display(_df)
    
def IBES_GA_predefined_variables(predefined):
    if predefined == "Yes":
        widgets.interact(Get_IBES_GA_predefined, area = drop_down_IBES_GA_return);
    elif predefined == "No":
        widgets.interact(Get_IBES_GA, area = drop_down_IBES_GA_return);

In [88]:
widgets.interact(IBES_GA_predefined_variables, predefined = drop_down_predefined_variables);

interactive(children=(Dropdown(description='Use predefined ordered_mnemonic, IBESGA_fields, xl_index and xl_co…

Now let's see our data-frame in the list:

In [89]:
DSWS_IBES_GA[0]

PE-Ratio  \
                                                                                                                              Fiscal Year   
                                                                                                                                      FY0   
Energy                     Energy                                         Energy                                         M1E1     460.023   
                                                                                                                         M2E2     460.023   
                                                                          Energy Equipment & Services                    M3ES      48.738   
                                                                          Oil, Gas & Consumable Fuels                    M3OG     754.708   
Materials                  Materials                                      Materials                                      M1M1      24.337   
                                                                                                                         M2M2      24.337   
                                                                          Chemicals                                      M3CH      30.134   
                                                                          Construction Materials                         M3CM      20.782   
                                                                          Containers & Packaging                         M3CT      21.836   
                                                                          Metals & Mining                                M3MM      20.305   
                                                                          Paper & Forest Products                        M3PF      21.168   
Industrials                Industrials                                    Industrials                                    M1ID      38.012   
                           Capital Goods                                  Capital Goods                                  M2CG      35.588   
                                                                          Aerospace & Defense                            M3AD      60.095   
                                                                          Building Products                              M3BP      27.988   
                                                                          Construction & Engineering                     M3CN      29.554   
                                                                          Electronic Equipment & Instruments             M3EI      34.149   
                                                                          Industrial Conglomerates                       M3IC       30.95   
                                                                          Machinery                                      M3MC      32.562   
                                                                          Trading Companies & Distributors Industry      M3TC      25.016   
                           Commercial Services & Supplies                 Commercial Services & Supplies                 M2C2      33.587   
                                                                          Commercial Services & Supplies                 M3C3      35.924   
                           Transportation                                 Transportation                                 M2TR      53.824   
                                                                          Air Freight & Couriers                         M3AF      23.481   
                                                                          Airlines                                       M3AL         NaN   
                                                                          Marine                                         M3MA      22.851   
                                        

## Conclusion

One can easily use interactive widgets to intuitively get dsws IBES GA data using our Python ```Get_IBES_GA``` function created above. We can even generate excel sheets replicating the 'Datastream IBES Global Aggregate MSCI.xlsm' file. However, this is only Part 1; I will attempt to see the best ways in which the above can be used to extract insights, possibly in graphical ways. If you have a workflow in mind for which the above would be useful, I would be happy to hear about it. Please do not hesitate to submit your propositions to jonathan.legrand@refinitiv.com

## References

* [Pivottable.js](https://towardsdatascience.com/two-essential-pandas-add-ons-499c1c9b65de)
* [Dashboarding with Jupyter Notebooks, Voila and Widgets | SciPy 2019 | M. Breddels and M. Renou](https://www.youtube.com/watch?v=VtchVpoSdoQ)

In [ ]:
import DatastreamDSWS as dsws
ds = dsws.Datastream(username = "insert dsws username here", password = "insert dsws password here")
from datetime import datetime
import warnings
from datetime import date
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

xl_regions = pd.DataFrame(data = {'Region': {0: 'EAFE', 1: 'EAFE + Canada',2: 'EAFE-ex-UK',3: 'EASEA (EAFE-ex-Japan)',4: 'EM (Emerging Markets)',5: 'EM Asia',6: 'EM Eastern Europe',7: 'EM Europe',8: 'EM Europe + Middle East',9: 'EM Europe, Middle East & Africa',10: 'EM Far East',11: 'EM Latin America',12: 'EMU (Euro)',13: 'EMU (US Dollar)',14: 'EMU + UK',15: 'Europe',16: 'Europe-ex-EMU',17: 'Europe-ex-UK',18: 'Far East',19: 'G7 Index',20: 'Kokusai (World-ex-Japan)',21: 'Nordic Countries',22: 'North America',23: 'Pacific',24: 'Pacific-ex-Japan',25: 'World',26: 'World-ex-Australia',27: 'World-ex-EMU',28: 'World-ex-Europe',29: 'World-ex-UK',30: 'World-ex-USA'},'Mnemonic': {0: 'EA',1: 'FC',2: 'EX',3: 'AS',4: 'EF',5: 'MI',6: 'EB',7: 'ME',8: 'UE',9: 'EM',10: 'MP',11: 'ML',12: 'MU',13: 'M$',14: 'MK',15: 'UR',16: 'EE',17: 'UX',18: 'FE',19: 'G7',20: 'KK',21: 'NC',22: 'NA',23: 'PC',24: 'PX',25: 'DW',26: 'WA',27: 'WE',28: 'WC',29: 'WK',30: 'WS'},'Code': {0: '@:M1EAFE',1: '@:M1EAFEC',2: '@:M1EAFEU',3: '@:M1EASEA',4: '@:M2EMG',5: '@:M2ASIA',6: '@:M2EEUR',7: '@:M2EUR',8: '@:M2EURME',9: '@:M2EMEA',10: '@:M2FARE',11: '@:M2LATAM',12: '@:MUMSCI',13: '@:M1EMU',14: '@:MKMSCI',15: '@:M1EROP',16: '@:EEMSCI',17: '@:M1EURXU',18: '@:M1FARE',19: '@:G7MSCI',20: '@:M1KOKUS',21: '@:M1NORD',22: '@:M1NAMER',23: '@:M1PAC',24: '@:M1PACXJ',25: '@:M1WRLD',26: '@:WAMSCI',27: '@:WEMSCI',28: '@:WCMSCI',29: '@:M1WLDXU',30: '@:M1WLDXA'}})
xl_countries = pd.DataFrame(data = {'Region': {0: 'Argentina',1: 'Australia',2: 'Australia',3: 'Austria',4: 'Belgium',5: 'Brazil',6: 'Canada',7: 'Chile',8: 'China',9: 'Colombia',10: 'Czech Republic',11: 'Denmark',12: 'Egypt',13: 'Finland',14: 'France',15: 'Germany',16: 'Greece',17: 'Hong Kong',18: 'Hungary',19: 'India',20: 'Indonesia',21: 'Ireland',22: 'Israel',23: 'Italy',24: 'Japan',25: 'Jordan',26: 'Korea',27: 'Malaysia',28: 'Mexico',29: 'Morocco',30: 'Netherlands',31: 'New Zealand',32: 'Norway',33: 'Pakistan',34: 'Peru',35: 'Philippines',36: 'Poland',37: 'Portugal',38: 'Russia',39: 'Singapore',40: 'South Africa',41: 'Spain',42: 'Sri Lanka',43: 'Sweden',44: 'Switzerland',45: 'Taiwan',46: 'Thailand',47: 'Turkey',48: 'United Kingdom',49: 'USA',50: 'Venezuela'},'Mnemonic': {0: 'AR',1: 'AU',2: 'AU',3: 'AT',4: 'BE',5: 'BR',6: 'CA',7: 'CL',8: 'CN',9: 'CO',10: 'CZ',11: 'DK',12: 'EG',13: 'FI',14: 'FR',15: 'DE',16: 'GR',17: 'HK',18: 'HU',19: 'IN',20: 'ID',21: 'IE',22: 'IL',23: 'IT',24: 'JP',25: 'JO',26: 'KR',27: 'MY',28: 'MX',29: 'MA',30: 'NL',31: 'NZ',32: 'NO',33: 'PK',34: 'PE',35: 'PH',36: 'PL',37: 'PT',38: 'RU',39: 'SG',40: 'SA',41: 'ES',42: 'LK',43: 'SE',44: 'SW',45: 'TW',46: 'TH',47: 'TR',48: 'UK',49: 'US',50: 'VE'}, 'Code': {0: '@:ARMSCIP',1: '@:MSCIP',2: '@:AUMSCIP',3: '@:OEMSCIP',4: '@:BGMSCIP',5: '@:BRMSCIP',6: '@:CNMSCIP',7: '@:CLMSCIP',8: '@:CHMSCIP',9: '@:CBMSCIP',10: '@:CZMSCIP',11: '@:DKMSCIP',12: '@:EYMSCIP',13: '@:FNMSCIP',14: '@:FRMSCIP',15: '@:BDMSCIP',16: '@:GRMSCIP',17: '@:HKMSCIP',18: '@:HNMSCIP',19: '@:INMSCIP',20: '@:IDMSCIP',21: '@:IRMSCIP',22: '@:ISMSCIP',23: '@:ITMSCIP',24: '@:JPMSCIP',25: '@:JOMSCIP',26: '@:KOMSCIP',27: '@:MYMSCIP',28: '@:MXMSCIP',29: '@:MCMSCIP',30: '@:NLMSCIP',31: '@:NZMSCIP',32: '@:NWMSCIP',33: '@:PKMSCIP',34: '@:PEMSCIP',35: '@:PHMSCIP',36: '@:POMSCIP',37: '@:PTMSCIP',38: '@:RSMSCIP',39: '@:SGMSCIP',40: '@:SAMSCIP',41: '@:ESMSCIP',42: '@:CYMSCIP',43: '@:SDMSCIP',44: '@:SWMSCIP',45: '@:TAMSCIP',46: '@:THMSCIP',47: '@:TKMSCIP',48: '@:UKMSCIP',49: '@:USMSCIP',50: '@:VEMSCIP'}})
xl_regions = xl_regions.append(xl_countries, ignore_index = True)

DSWS_IBES_GA = []

drop_down_IBES_GA_return = widgets.Dropdown(options = [""] + xl_regions["Region"].to_list(),value = "", disabled = False)

drop_down_predefined_variables = widgets.Dropdown(options = [""] + ["Yes", "No"], value = "", description = "Use predefined ordered_mnemonic, IBESGA_fields, xl_index and xl_columns2 variables?", disabled = False)

def Get_IBES_GA_predefined(area):
    >>>     _df = Get_IBES_GA(area)
    >>>     display(_df)
    >>> 
    >>> def IBES_GA_predefined_variables(predefined):
    >>>     if predefined == "Yes":
    >>>         widgets.interact(Get_IBES_GA_predefined, area = drop_down_IBES_GA_return);
    >>>     elif predefined == "No":
    >>>         widgets.interact(Get_IBES_GA, area = drop_down_IBES_GA_return);
    >>> 
    >>> widgets.interact(IBES_GA_predefined_variables, predefined = drop_down_predefined_variables);
    >>> 
    >>> display(DSWS_IBES_GA[0])